In [21]:
#4.17
from triang import forsub, backsub, testcreate, testsolve
import numpy as np
from gauelim import gauelim
from gauelim_pivot import gauelim_pivot

if __name__ == "__main__":
    A = np.array([4., 4, 8, 4,
                  4, 5, 3, 7,
                  8, 3, 9, 9,
                  4, 7, 9, 5]).reshape(4,4)
    b = np.array([1., 2, 3, 4])

    print("Matrix A:\n", A)
    print("Vector b:", b)

    # Gaussian elimination without pivoting
    x1 = gauelim(A, b)
    print("\nGaussian elimination (no pivot):", x1)
    print("Check Ax:", A @ x1)
  
    # Gaussian elimination with pivoting
    x2 = gauelim_pivot(A, b)
    print("\nGaussian elimination (no pivot):", x2)
    print("Check Ax:", A @ x2)

print("""
Because pivoted and non-pivoted both failed to provide a solution, we can infer that
pivoting is helpful but is not a universal solution. It cannot solve a singular or
inconsistent system. It only makes the factorization numerically stable and reveals
a zero pivot which will then tell you that the matrix is rank deficient.

We can say that matrix A it is rank deficient and the system Ax=b is inconsistent
with no exact solution. """)

def gauelim_replace(inA, inbs, epsilon=1e-20):
    """Gaussian elimination with zero pivot replacement"""
    A = np.copy(inA)
    bs = np.copy(inbs)
    n = bs.size

    for j in range(n-1):
        # Check if pivot is zero and replace if necessary
        if abs(A[j, j]) < 1e-15:
            print(f"  Replacement: pivot A[{j},{j}] = {A[j,j]:.2e} replaced with {epsilon:.2e}")
            A[j, j] = epsilon
        
        for i in range(j+1, n):
            coeff = A[i, j] / A[j, j]
            A[i, j:] -= coeff * A[j, j:]
            bs[i] -= coeff * bs[j]

    xs = backsub(A, bs)
    return xs

x2 = gauelim_replace(A, b)
print("\nGaussian elimination (no pivot):", x3)
print("Check Ax:", A @ x3)


print("""
By replacing the zero pivot with a tiny number, there will be an answer to the problem
but there is a big error in roundoff and results will be practically meaningless. In
addition, it also covers or hides the real problem that the system is actually
inconsistent.
""")

Matrix A:
 [[4. 4. 8. 4.]
 [4. 5. 3. 7.]
 [8. 3. 9. 9.]
 [4. 7. 9. 5.]]
Vector b: [1. 2. 3. 4.]

Gaussian elimination (no pivot): [nan nan inf inf]
Check Ax: [nan nan nan nan]

Gaussian elimination (no pivot): [ nan  inf -inf -inf]
Check Ax: [nan nan nan nan]

Because pivoted and non-pivoted both failed to provide a solution, we can infer that
pivoting is helpful but is not a universal solution. It cannot solve a singular or
inconsistent system. It only makes the factorization numerically stable and reveals
a zero pivot which will then tell you that the matrix is rank deficient.

We can say that matrix A it is rank deficient and the system Ax=b is inconsistent
with no exact solution. 

Gaussian elimination (no pivot): [nan nan inf inf]
Check Ax: [nan nan nan nan]

By replacing the zero pivot with a tiny number, there will be an answer to the problem
but there is a big error in roundoff and results will be practically meaningless. In
addition, it also covers or hides the real problem th

In [25]:
import numpy as np

def build_cyclic_tridiag(n):
    A = np.zeros((n,n))
    b = np.ones(n)

    for i in range(n):
        A[i,i] = 4
        if i > 0:
            A[i,i-1] = -1
        if i < n-1:
            A[i,i+1] = -1
    # cyclic connections
    A[0,-1] = -1
    A[-1,0] = -1
    return A, b

def jacobi_cyclic(n, kmax=1000, tol=1e-5):
    x = np.zeros(n)   # initial guess
    for k in range(1, kmax+1):
        x_new = np.zeros(n)
        # first row
        x_new[0] = (1 + x[1] + x[-1]) / 4
        # interior rows
        for i in range(1, n-1):
            x_new[i] = (1 + x[i-1] + x[i+1]) / 4
        # last row
        x_new[-1] = (1 + x[0] + x[-2]) / 4

        # error check
        err = np.linalg.norm(x_new - x, ord=np.inf)
        if err < tol:
            return x_new, k
        x = x_new
    return x, kmax

if __name__ == "__main__":
    for n in [10, 20]:
        # Jacobi
        x_jacobi, iters = jacobi_cyclic(n)
        print(f"\nJacobi solution for n={n} after {iters} iterations:\n", x_jacobi)

        # Exact solution
        A, b = build_cyclic_tridiag(n)
        x_exact = np.linalg.solve(A, b)
        print(f"NumPy solution for n={n}:\n", x_exact)

        # Difference
        diff = np.linalg.norm(x_jacobi - x_exact, ord=np.inf)
        print(f"Max difference Jacobi vs NumPy (n={n}): {diff:.2e}")



Jacobi solution for n=10 after 16 iterations:
 [0.49999237 0.49999237 0.49999237 0.49999237 0.49999237 0.49999237
 0.49999237 0.49999237 0.49999237 0.49999237]
NumPy solution for n=10:
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
Max difference Jacobi vs NumPy (n=10): 7.63e-06

Jacobi solution for n=20 after 16 iterations:
 [0.49999237 0.49999237 0.49999237 0.49999237 0.49999237 0.49999237
 0.49999237 0.49999237 0.49999237 0.49999237 0.49999237 0.49999237
 0.49999237 0.49999237 0.49999237 0.49999237 0.49999237 0.49999237
 0.49999237 0.49999237]
NumPy solution for n=20:
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5]
Max difference Jacobi vs NumPy (n=20): 7.63e-06
